<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Введение" data-toc-modified-id="Введение-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Введение</a></span></li><li><span><a href="#Цель-исследования" data-toc-modified-id="Цель-исследования-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Цель исследования</a></span></li><li><span><a href="#План-работы" data-toc-modified-id="План-работы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>План работы</a></span></li><li><span><a href="#Описание-выгруженной-таблицы" data-toc-modified-id="Описание-выгруженной-таблицы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Описание выгруженной таблицы</a></span></li><li><span><a href="#Шаг-1-Подключение-к-базе-данных" data-toc-modified-id="Шаг-1-Подключение-к-базе-данных-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Шаг 1 Подключение к базе данных</a></span></li><li><span><a href="#Шаг-2-Выгрузка-данных" data-toc-modified-id="Шаг-2-Выгрузка-данных-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Шаг 2 Выгрузка данных</a></span><ul class="toc-item"><li><span><a href="#Формирование-SQL-запроса" data-toc-modified-id="Формирование-SQL-запроса-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Формирование SQL-запроса</a></span></li><li><span><a href="#Выгрузка-данных-в-формате-csv" data-toc-modified-id="Выгрузка-данных-в-формате-csv-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Выгрузка данных в формате csv</a></span></li></ul></li><li><span><a href="#Дашборд" data-toc-modified-id="Дашборд-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Дашборд</a></span></li><li><span><a href="#Презентация" data-toc-modified-id="Презентация-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Презентация</a></span></li><li><span><a href="#Общий-вывод-по-исследованию" data-toc-modified-id="Общий-вывод-по-исследованию-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Общий вывод по исследованию</a></span></li></ul></div>

# Оценка текущего уровня лояльности клиентов телекоммуникационной компании

## Введение

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

## Цель исследования

По ответам пользователей подготовить дашборд для определения текущего уровня потребильской лояльности. Для оценки результатов опроса клиенты были разделены на три группы согласно европейским стандартам:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).
Итоговое значение NPS рассчитывается по формуле: `% «сторонников»` - `% «критиков»`.

## План работы

1. Подключиться к базе данных с результатами опроса;
2. Выгрузить нужные данные;
3. Создать интерактивный дашборд в Tableau;
4. При помощи дашборда ответить на следующие вопросы заказчика: Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе? Какие группы пользователей наиболее лояльны к сервису? Какие менее? Какой общий NPS среди всех опрошенных? Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?
5. Подготовить презентацию для заказчика с овтетами на вопросы;
6. Написать общий вывод.

## Описание выгруженной таблицы

1. `user_id` - Идентификатор клиента;
2. `lt_day` - Количество дней «жизни» клиента;
3. `is_new` - Поле хранит информацию о том, является ли клиент новым;
4. `age` - Возраст;
5. `gender_segment` - Пол;
6. `os_name` - Тип операционной системы;
7. `cpe_type_name` - Тип устройства;
8. `country` - Страна проживания;
9. `city` - Город проживания;
10. `age_segment` - Возрастной сегмент;
11. `traffic_segment` - Сегмент по объёму потребляемого трафика;
12. `lifetime_segment` - Сегмент по количеству дней «жизни»;
13. `nps_score` - Оценка клиента в NPS-опросе;
14. `nps_group` - Поле хранит информацию о том, к какой группе относится оценка клиента в опросе;


## Шаг 1 Подключение к базе данных

In [1]:
# Основные библиотеки, которые нам необходимы:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# путь к БД на вашем компьютере (например, в той же папке, что и тетрадь)
path_to_db_local = 'C:/Users/MSI/Desktop/NPS report/telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Шаг 2 Выгрузка данных

### Формирование SQL-запроса

In [3]:
# SQL-запрос нужных данных
query = """
SELECT u.user_id, u.lt_day, 
       CASE
           WHEN u.lt_day <= 365 THEN true
           WHEN u.lt_day > 365 THEN false
       END AS is_new,
       u.age, 
       CASE
           WHEN u.gender_segment = 1 THEN 'женщина'
           WHEN u.gender_segment = 0 THEN 'мужчина'
       END AS gender_segment,
       u.os_name, 
       u.cpe_type_name, loc.country, loc.city, 
       ag.title AS age_segment, 
       tg.title AS traffic_segment,
       lg.title AS life_time_segment, 
       u.nps_score,
       CASE
           WHEN u.nps_score >= 9 THEN 'сторонники'
           WHEN u.nps_score >= 7 AND u.nps_score < 9 THEN 'нейтралы'
           WHEN u.nps_score < 7 THEN 'критики'
        END AS nps_group
FROM user AS u
INNER JOIN location AS loc ON loc.location_id=u.location_id
INNER JOIN age_segment AS ag ON ag.age_gr_id=u.age_gr_id
INNER JOIN traffic_segment AS tg ON tg.tr_gr_id=u.tr_gr_id
INNER JOIN lifetime_segment AS lg ON lg.lt_gr_id=u.lt_gr_id
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,life_time_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,0,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,0,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   user_id            502493 non-null  object 
 1   lt_day             502493 non-null  int64  
 2   is_new             502493 non-null  int64  
 3   age                501939 non-null  float64
 4   gender_segment     501192 non-null  object 
 5   os_name            502493 non-null  object 
 6   cpe_type_name      502493 non-null  object 
 7   country            502493 non-null  object 
 8   city               502493 non-null  object 
 9   age_segment        502493 non-null  object 
 10  traffic_segment    502493 non-null  object 
 11  life_time_segment  502493 non-null  object 
 12  nps_score          502493 non-null  int64  
 13  nps_group          502493 non-null  object 
dtypes: float64(1), int64(3), object(10)
memory usage: 53.7+ MB


### Выгрузка данных в формате csv

In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,life_time_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,0,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,0,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники


## Дашборд

Доступно по ссылке: https://public.tableau.com/views/Draft_16842223943300/Dashboard1?:language=en-US&:display_count=n&:origin=viz_share_link

## Презентация

Доступно по ссылке: https://disk.yandex.ru/i/FYts2ITt2ihl1A

## Общий вывод по исследованию

В исследовании приняли участие 501152 человек, среди которых 54% женщин и 46% мужчин. Согласно подсчетам по полученным от Заказчика данным наиболее лояльными к услугам являются женщины в возрасте 45 лет из крупных городов. Также стоит отметить, что новым клиентам (у тех, которых с момента значимой покупки произошло не более 365 дней) предоставленные услуги нравятся. Новых клиентов намного меньше, чем старых, которые составляют более 80% опрашиваемых. Это означает, что недовольство давних клиентов увеличивается, а значит во многом требуется пересмотреть некоторые услуги в пользу данной группы клиентов для дальнейшего увеличения лояльности.